In [71]:
import pandas as pd
import warnings


In [119]:
def wrangle(file):

    #read csv
    df = pd.read_csv(file)

    # columns to drop due to high missing values
    cols_to_drop = ['Year', 'Class', 'Adm1', 'Adm6', 'Geo1', 'econ4', 'Adm3', 'Adm4', 'Adm5', 'Econ0', 'econ1', 'econ2', 'econ3', 'FndSrce1', 'function1', 'function2']
      
    #drop columns
    df.drop(columns = cols_to_drop, inplace=True)

    #remove empty spaces
    df.columns = df.columns.str.strip()

    #Remove characters from target numerical columns
    num_cols = ['Estimates', 'Revised', 'Executed']
    for col in num_cols:
        df[col] = df[col].astype(str)
        df[col] = df[col].str.replace(r'[^\d.-]', '', regex=True) 
        df[col] = df[col].replace(r'^\s*$', 'NaN', regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce') 
    
    #handle missing data
    df.dropna(subset=['Estimates'], inplace=True) # Drop rows where "Estimates" is missing

    # Fill missing "Revised" and "Executed" values with 0
    df[['Revised', 'Executed']] = df[['Revised', 'Executed']].fillna(0) 

    #change column names
    df.columns = ['Ministry', 'Estimates', 'Revised', 'Executed']
    
    #remove ministry code 
    df['Ministry'] = df['Ministry'].str.replace(r'^\d+\s*', '', regex=True)
    

    #return dataframe
    return df

In [120]:
df = wrangle('kenya-boost-2006-2018.csv')
df.head()


/tmp/ipykernel_2753/2622435060.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,Ministry,Estimates,Revised,Executed
0,Ministry of Agriculture,18318600.0,14062700.0,5893860.0
1,Ministry of Agriculture,12799450.0,9599587.0,3312808.0
2,Ministry of Agriculture,7218400.0,5652150.0,2911400.0
3,Ministry of Agriculture,10025827.0,8126744.0,4501504.0
4,Ministry of Agriculture,500000.0,1130625.0,95000.0


In [121]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 254963 entries, 0 to 263740
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Ministry   254963 non-null  object 
 1   Estimates  254963 non-null  float64
 2   Revised    254963 non-null  float64
 3   Executed   254963 non-null  float64
dtypes: float64(3), object(1)
memory usage: 9.7+ MB
